From our nlp course project
https://github.com/Aaalan-Zhang/nlp-from-scratch-assignment-fall2024/blob/main/crawler/tests/0_webpage_parsing.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.listdir("./drive/MyDrive/Proj/News_fetcher")

os.chdir("./drive/MyDrive/Proj/News_fetcher")

Mounted at /content/drive


# Crawler

In [ ]:
# # === Install Chrome and ChromeDriver v137 ===
# !apt-get remove -y chromium-browser google-chrome-stable || true
# !rm -rf /usr/bin/google-chrome /usr/bin/chromedriver137

# !wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.119/linux64/chrome-linux64.zip
# !unzip -q chrome-linux64.zip
# !mv chrome-linux64/chrome /usr/bin/google-chrome

# !wget -q https://storage.googleapis.com/chrome-for-testing-public/137.0.7151.119/linux64/chromedriver-linux64.zip
# !unzip -q chromedriver-linux64.zip
# !mv chromedriver-linux64/chromedriver /usr/bin/chromedriver137
# !chmod +x /usr/bin/google-chrome /usr/bin/chromedriver137

In [ ]:
%pip install -q google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typeguard 4.4.3 requires typing_extensions>=4.14.0, but you have typing-extensions 4.13.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_mach

## Original Crawler from 711

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import warnings
from urllib.parse import urljoin
from urllib3.exceptions import InsecureRequestWarning

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
]
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Define headers for requests
headers = {'User-Agent': random.choice(user_agents)}
session = requests.Session()

# Fetch the page text with retry mechanism
def fetch_page_text(url, retries=3, timeout=5):
    attempt = 0
    while attempt < retries:
        try:
            # Send a GET request to the URL with a timeout
            response = session.get(url, timeout=timeout, headers=headers, verify=False)
            response.raise_for_status()  # Raise an error for bad responses

            # Parse the page content
            soup = BeautifulSoup(response.content, 'lxml')

            # Extract and return the text from the page and the soup object
            return soup.get_text(separator='\n', strip=True), soup

        except requests.exceptions.RequestException as e:
            attempt += 1
            time.sleep(1)  # Optional: wait before retrying

        if attempt == retries:
            return None, None

# Extract all sublinks (anchor tags) and handle relative URLs
def extract_sublinks(soup, base_url):
    sublinks = []
    for link in soup.find_all('a', href=True):
        sublink = link['href']
        # Convert relative URLs to absolute URLs using urljoin
        full_url = urljoin(base_url, sublink)
        sublinks.append(full_url)
    return sublinks

# Crawl through the URLs and fetch the data with retry
def crawl_urls(url_list):
    results = {}
    sublinks_data = []  # List to store (parent_url, sublink) pairs
    for index, url in enumerate(url_list):
        print(f"Fetching: {url}")
        text, soup = fetch_page_text(url)
        if text and soup:
            # Store the crawled text
            results[url] = text
            # Extract sublinks
            sublinks = extract_sublinks(soup, url)
            for sublink in sublinks:
                sublinks_data.append((url, sublink))
        else:
            print(f"Failed to parse URL at index: {index}, URL: {url}")
    return results, sublinks_data

# Save the crawled text data to .txt files
def save_crawled_data(crawled_data, urls):
    for index, url in enumerate(urls):
        # Fetch the text corresponding to the URL from the crawled_data dictionary
        text = crawled_data.get(url, "")

        # Remove all newline characters from the text
        cleaned_text = text.replace('\n', ' ')

        # Define the file name with the index from the URLs list
        output_file = f"/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_text_data/{index}.txt"

        # Save the cleaned text to the file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(cleaned_text)

# Save sublinks to a CSV file
def save_sublinks_to_csv(sublinks_data, output_csv):
    df = pd.DataFrame(sublinks_data, columns=['Parent URL', 'Sublink'])
    df.to_csv(output_csv, index=False)

if __name__ == "__main__":
    file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/raw/raw_csv_data/data_source.csv'
    data = pd.read_csv(file_path)

    # Extract non-empty URLs from the 'Source URL' column
    urls = data[data['Select'] == 'Webpage']['Source URL'].dropna().unique()

    # Start crawling the URLs
    crawled_data, sublinks_data = crawl_urls(urls)

    # Save the crawled text data
    save_crawled_data(crawled_data, urls)

    # Save sublinks to a CSV file
    sublinks_output_file = "/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_sublinks.csv"
    save_sublinks_to_csv(sublinks_data, sublinks_output_file)

    print("Crawling complete!")

## 参考消息

In [ ]:
from datetime import datetime
url = "https://www.cankaoxiaoxi.com/#/detailsPage/tt/329d19f3dfae4b50b524f428cd62637c/1/2025-06-18%2009:46?childrenAlias=undefined"
prefix = "/content/drive/MyDrive/Proj/News_fetcher/"


# Get today's date
suffix = datetime.today().strftime("%Y-%m-%d")
suffix

'2025-06-19'

### Main

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import time
# import random
# import warnings
# from urllib.parse import urljoin
# from urllib3.exceptions import InsecureRequestWarning
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
# import time

# user_agents = [
#     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#     'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#     'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
# ]
# warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# # Define headers for requests
# headers = {'User-Agent': random.choice(user_agents)}
# session = requests.Session()

# # Fetch the page text with retry mechanism
# def fetch_page_text(url, retries=3, timeout=5):
#     attempt = 0
#     while attempt < retries:
#         try:
#             # Send a GET request to the URL with a timeout
#             response = session.get(url, timeout=timeout, headers=headers, verify=False)
#             response.raise_for_status()  # Raise an error for bad responses

#             # Parse the page content
#             soup = BeautifulSoup(response.content, 'lxml')
#             print("response is :")
#             print(response.content)
#             # Extract and return the text from the page and the soup object
#             return soup.get_text(separator='\n', strip=True), soup

#         except requests.exceptions.RequestException as e:
#             attempt += 1
#             time.sleep(1)  # Optional: wait before retrying

#         if attempt == retries:
#             return None, None

# # def fetch_page_text_selenium(url, retries=3, timeout=5):
# #     # Setup headless Chrome
# #     options = Options()
# #     options.add_argument('--headless')
# #     options.add_argument('--disable-gpu')

# #     driver = webdriver.Chrome(options=options)

# #     # Open page
# #     driver.get("https://your_url_here.com")
# #     time.sleep(3)  # wait for JS to load; adjust as needed

# #     # Get rendered HTML
# #     html = driver.page_source
# #     soup = BeautifulSoup(html, 'lxml')

# #     # Now you can extract dynamically loaded content
# #     print(soup.prettify())

# #     driver.quit()

# # Extract all sublinks (anchor tags) and handle relative URLs
# def extract_sublinks(soup, base_url):
#     sublinks = []
#     for link in soup.find_all('a', href=True):
#         sublink = link['href']
#         # Convert relative URLs to absolute URLs using urljoin
#         full_url = urljoin(base_url, sublink)
#         sublinks.append(full_url)
#     return sublinks

# # Crawl through the URLs and fetch the data with retry
# def crawl_urls(url_list):
#     results = {}
#     sublinks_data = []  # List to store (parent_url, sublink) pairs
#     for index, url in enumerate(url_list):
#         print(f"Fetching: {url}")
#         text, soup = fetch_page_text(url)
#         if text and soup:
#             # Store the crawled text
#             results[url] = text
#             # Extract sublinks
#             sublinks = extract_sublinks(soup, url)
#             for sublink in sublinks:
#                 sublinks_data.append((url, sublink))
#         else:
#             print(f"Failed to parse URL at index: {index}, URL: {url}")
#     return results, sublinks_data

# # Save the crawled text data to .txt files
# def save_crawled_data(crawled_data, urls, prefix):
#     for index, url in enumerate(urls):
#         # Fetch the text corresponding to the URL from the crawled_data dictionary
#         text = crawled_data.get(url, "")

#         # Remove all newline characters from the text
#         cleaned_text = text.replace('\n', ' ')

#         # Define the file name with the index from the URLs list
#         output_file = f"{prefix}data/crawled_data/crawled_text_data/{index}.txt"
#         output_dir = os.path.dirname(output_file)

#         # Check if directory exists, and create if not
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         # Save the cleaned text to the file
#         with open(output_file, 'w', encoding='utf-8') as file:
#             file.write(cleaned_text)

# # Save sublinks to a CSV file
# def save_sublinks_to_csv(sublinks_data, output_csv):
#     df = pd.DataFrame(sublinks_data, columns=['Parent URL', 'Sublink'])
#     df.to_csv(output_csv, index=False)

# if __name__ == "__main__":
#     # file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/raw/raw_csv_data/data_source.csv'
#     # data = pd.read_csv(file_path)

#     # # Extract non-empty URLs from the 'Source URL' column
#     # urls = data[data['Select'] == 'Webpage']['Source URL'].dropna().unique()

#     url = "https://www.cankaoxiaoxi.com/#/detailsPage/tt/329d19f3dfae4b50b524f428cd62637c/1/2025-06-18%2009:46?childrenAlias=undefined"
#     prefix = "/content/drive/MyDrive/Proj/News_fetcher/"

#     urls = [url]
#     # Start crawling the URLs
#     crawled_data, sublinks_data = crawl_urls(urls)

#     # Save the crawled text data
#     save_crawled_data(crawled_data, urls, prefix)

#     # Save sublinks to a CSV file
#     sublinks_output_file = prefix + "data/crawled_data/crawled_sublinks.csv"
#     save_sublinks_to_csv(sublinks_data, sublinks_output_file)

#     print("Crawling complete!")

Fetching: https://www.cankaoxiaoxi.com/#/detailsPage/tt/329d19f3dfae4b50b524f428cd62637c/1/2025-06-18%2009:46?childrenAlias=undefined
response is :
b'<!doctype html><html><head><meta charset="UTF-8"/><meta name="keywords" content=""><meta name="description" content=""><meta name="apple-mobile-web-app-capable" content="yes"/><meta name="apple-touch-fullscreen" content="yes"/><meta name="apple-mobile-web-app-status-bar-style" content="black"/><title>\xe5\x8f\x82\xe8\x80\x83\xe6\xb6\x88\xe6\x81\xaf</title><script defer="defer" src="/statics/ckxx-website/static/js/main.4651e5cf.js"></script><link href="/statics/ckxx-website/static/css/main.dd0b7dd0.css" rel="stylesheet"></head><script>window._czc = window._czc || []; //\xe7\xbb\x9f\xe8\xae\xa1\xe7\x94\xa8\xe5\x85\xa8\xe5\xb1\x80\xe5\x8f\x98\xe9\x87\x8f\n    let targetWidth = 1400;\n    // let scale = document.documentElement.clientWidth / targetWidth;\n    let scale = window.screen.width / targetWidth;\n    const viewport = document.create

<IPython.core.display.Javascript object>

### Workable for 参考消息


In [ ]:
def wait_for_stable_page(driver, timeout=10, interval=1.5):
    """Waits for the whole text content to stop changing."""
    stable_counter = 0
    prev_text = ""
    start_time = time.time()

    while time.time() - start_time < timeout:
        try:
            html = driver.page_source
            curr_text = html.strip()
        except:
            curr_text = ""

        if curr_text == prev_text and len(curr_text) > 0:
            stable_counter += 1
            if stable_counter >= 2:  # Stable twice in a row
                return curr_text
        else:
            stable_counter = 0
        prev_text = curr_text
        time.sleep(interval)
        # print(len(curr_text))
    return
    raise TimeoutException("detailsPage content did not stabilize in time.")

def safe_get(driver, url, max_retries=5, wait_between=3):
    for attempt in range(max_retries):
        try:
            driver.get(url)
            return  # success
        except TimeoutException:
            print(f"[Attempt {attempt+1}] Timeout loading: {url}")
            time.sleep(wait_between)
        except (WebDriverException, InvalidSessionIdException) as e:
            print(f"[{attempt+1}] WebDriver error: {e}. Restarting driver...")
            try:
                driver.quit()
            except Exception:
                pass
            driver = get_fresh_driver()
            time.sleep(2)
    raise TimeoutException(f"Failed to load {url} after {max_retries} attempts.")

def get_fresh_driver():
    try:
        print("Initializing Chromedriver")
        driver = gs.Chrome()
        time.sleep(2)  # Let browser settle
        return driver
    except Exception as e:
        print("Driver failed to initialize:", e)
        return None

In [ ]:
def restart_driver_and_get(url, retries=3):
    for attempt in range(retries):
        driver = get_fresh_driver()
        if driver is None:
            continue
        try:
            driver.get(url)
            return driver
        except WebDriverException as e:
            print(f"[{attempt+1}] Error loading {url}: {e}")
            try:
                driver.quit()
            except:
                pass
            time.sleep(2)
    print("All attempts failed.")
    return None

In [ ]:
import pandas as pd

# Assume these are your lists
# sublinks = ['https://example.com/page1', 'https://example.com/page2', ...]
# contents = ['Text content of page 1...', 'Text content of page 2...', ...]

# Combine into a DataFrame
def save_to_csv(sublinks, contents, file_name, tags=None):
    df = pd.DataFrame({
        'ind': range(len(sublinks)),
        'url': sublinks,
        'content': contents
    })
    if tags:
      df['tag'] = tags
    # Save to CSV
    df.to_csv(f'{prefix}data/crawled_data/{suffix}_{file_name}', index=False, encoding='utf-8-sig')  # Use utf-8-sig for Chinese

def load_csv(file_name):
    return pd.read_csv(f'{prefix}data/crawled_data/{suffix}_{file_name}', encoding='utf-8-sig')

### Index Page

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    WebDriverException,
    TimeoutException,
    NoSuchElementException,
    InvalidSessionIdException
)
from bs4 import BeautifulSoup
import time
import google_colab_selenium as gs

In [ ]:
index_url = "https://www.cankaoxiaoxi.com/#/index"
sublinks = []
contents = []



# # Parse after content loads
# html = driver.page_source
# soup = BeautifulSoup(html, "lxml")

# # Print readable content
# print("Title:", driver.title)
# print(soup.get_text()[:10000])  # preview 1000 chars of text

cankaoxiaoxi_index_visiting(sublinks, contents, index_url)
# driver.quit()
save_to_csv(sublinks, contents, "index_contents.csv")

NameError: name 'cankaoxiaoxi_index_visiting' is not defined

### generalColumns


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    WebDriverException,
    TimeoutException,
    NoSuchElementException,
    InvalidSessionIdException
)
from bs4 import BeautifulSoup
import time
import google_colab_selenium as gs

In [ ]:
sublinks = []
contents = []
tags = []
generalColumnsDict = {
    '观点': "https://www.cankaoxiaoxi.com/#/generalColumns/guandian",
}
for k, v in generalColumnsDict.items():
    cankaoxiaoxi_generalColumns_visiting(sublinks, contents, tags, k, v, "templateModule")


save_to_csv(sublinks, contents, "index_contents.csv", tags)


In [ ]:
contents

['参考消息第一关注中国国际观点锐参考体育健康科技应用文化旅游参考漫谈参考智库军事民族品牌参考人物产经地方译名文娱我的位置：首页>观点>新闻详情澳前外长：“澳大利亚应期待澳英美联盟崩溃”2025-06-18 16:23:1918573分享到：用微信扫描二维码参考消息网6月18日报道 世界报业辛迪加网站6月16日刊登题为《澳大利亚应期待澳英美联盟的崩溃》的文章，作者为澳大利亚前外交部长加雷思·埃文斯。内容编译如下：根据2021年达成的澳英美联盟合作协议，美国和英国将在30年内向澳大利亚提供至少八艘核动力潜艇。目前，美国国防部正在对该协议进行审查。如果协议破裂，那些乐见三国同盟深化的人士——尤其是盼着澳大利亚给资金短缺、效率低下的美英海军造船厂输送数千亿美元资金的人——不出意外会陷入焦虑。但对澳大利亚而言，澳英美联盟的瓦解可喜可贺。毕竟，关于协议承诺的潜艇能否按时交付，始终存在不确定性。根据计划，美国应从2032年起交付三艘(或五艘)“弗吉尼亚”级潜艇，另外五艘新设计的“澳英美联盟”级核潜艇则计划于本世纪40年代初服役。但美英两国受本国潜艇建造目标的影响，工业产能已处于饱和状态，且双方均明确拥有协议退出权。一些分析人士认为，美国国防部此次审查不过是特朗普式的敲诈伎俩，意在从澳大利亚榨取更多财政投入。这种解读虽能让部分澳大利亚人感到宽慰，但其实是错误的。即使一切顺利——从“弗吉尼亚”级潜艇的交付到英国新潜艇的建造，既没有遇到人力资源瓶颈，也没有发生成本超支——澳大利亚仍需等待数十年才能接收最后一艘潜艇。但澳大利亚老旧的“柯林斯”级潜艇舰队已进入退役期，这一时间表构成了严峻挑战：在此期间，我们该如何填补能力缺口？澳大利亚本应在一开始就通过成本效益分析否决该项目，但政治领导人急于达成这项协议，并没有认真审查内容。即使承认核动力潜艇在速度与续航能力上有着显著优势，且乐观地假定其水下隐蔽性在服役期内不会受到技术挑战，最终的舰队规模仍明显不符合国防需求。考虑到常规操作限制，澳大利亚任何时候最多只能部署两艘此类潜艇。在这种情况下，究竟能完成多少情报收集、群岛要塞保护、海上航道护卫，甚至远程威慑任务？此外，该项目天文数字般的成本将导致澳大利亚难以获取其他关键能力，如先进无人机、导弹、飞机和网络防御系统，而这些能力正在重新定义现代战争。正如澳大利亚前总理保罗·基廷所言，美国若退出协议，将

In [ ]:
save_to_csv(sublinks, contents, "generalColumns_sublink_contents.csv", tags)

#### simulate


##### index

In [ ]:
def cankaoxiaoxi_index_visiting(sublinks, contents, url):
    # driver = get_fresh_driver()
    # safe_get(driver, url)
    driver = restart_driver_and_get(url)
    wait_for_stable_page(driver)


    num_articles = len(driver.find_elements(By.CLASS_NAME, "clickPart"))
    original_url = driver.current_url
    cnt = 0
    while cnt < num_articles:
        try:
            # print("Enter Index Page")
            original_window = driver.current_window_handle
            cards = driver.find_elements(By.CLASS_NAME, "clickPart")
            # print(driver.current_url)
            cards[cnt].click()

            for handle in driver.window_handles:
                if handle != original_window:
                    # print("Switching!!")
                    driver.switch_to.window(handle)
                    break
            wait_for_stable_page(driver)
            article_url = driver.current_url
            sublinks.append(article_url)
            # print(article_url)
            # Scrape the article
            soup = BeautifulSoup(driver.page_source, "lxml")
            # article = soup.find("div", class_="detailsPage")
            # print(article.get_text(strip=True) if article else "No article found")
            article = soup.get_text()
            print(driver.title)
            print(article)
            contents.append(article)

            # Go back to the list page
            driver.close()
            driver.switch_to.window(original_window)
            # print(driver.current_url)

        except Exception as e:
            print(e)
            try:
                # safe_get(driver, url)
                driver = restart_driver_and_get(url)
                wait_for_stable_page(driver)
            except:
                print(f"[{cnt}] Retry failed. Aborting.")
                break
        cnt += 1
    driver.quit()

用更多新闻来截断
From 首页>新闻详情 to 更多新闻

##### generalColumns

In [ ]:
def cankaoxiaoxi_generalColumns_visiting(sublinks, contents, tags, tag, col_url, target_class, load_button_clicks=3):
    # driver = get_fresh_driver()
    # safe_get(driver, url)
    driver = restart_driver_and_get(col_url)
    wait_for_stable_page(driver)


    load_button = driver.find_elements(By.CLASS_NAME, "generalColumns-loadMore")
    if load_button:
      load_button = load_button[0]
      for _ in range(load_button_clicks):
        load_button.click()
        wait_for_stable_page(driver)
        load_button = driver.find_elements(By.CLASS_NAME, "generalColumns-loadMore")[0]
        # print(len(driver.find_elements(By.CLASS_NAME, target_class)))

    num_articles = len(driver.find_elements(By.CLASS_NAME, target_class))
    cnt = 0
    while cnt < num_articles:
        try:
            print("Enter Page")
            original_window = driver.current_window_handle
            cards = driver.find_elements(By.CLASS_NAME, target_class)
            print(driver.current_url)
            cards[cnt].click()

            for handle in driver.window_handles:
                if handle != original_window:
                    print("Switching!!")
                    driver.switch_to.window(handle)
                    break
            wait_for_stable_page(driver)
            article_url = driver.current_url
            sublinks.append(article_url)
            tags.append(tag)
            # Scrape the article
            soup = BeautifulSoup(driver.page_source, "lxml")
            # article = soup.find("div", class_="detailsPage")
            # print(article.get_text(strip=True) if article else "No article found")
            article = soup.get_text()
            print(driver.title)
            print(article)
            contents.append(article)


            # Go back to the list page
            driver.close()
            driver.switch_to.window(original_window)
            # print(driver.current_url)

        except Exception as e:
            print(e)
            try:
                # safe_get(driver, url)
                driver = restart_driver_and_get(url)
                wait_for_stable_page(driver)
            except:
                print(f"[{cnt}] Retry failed. Aborting.")
                break
        cnt += 1
    driver.quit()

### post process of contents

In [ ]:
load_csv("generalColumns_sublink_contents.csv")

#### clickpart 不行，他是用event listener处理的 需要simulate

In [ ]:
cards = driver.find_elements(By.CLASS_NAME, "clickPart")
for card in cards:
    print(card)
    onclick = card.get_attribute("onclick")
    if onclick:
        print("onclick JS:", onclick)
for card in soup.select(".clickPart"):
    print(1)
    a = card.find("a")
    if a and a.get("href"):
        print("Title:", card.select_one(".content_title").text.strip())
        print("URL:", a["href"])

<selenium.webdriver.remote.webelement.WebElement (session="dc738d92e277d753d945b6fb1ba3a802", element="f.C8121B418917C39CE0D66451E3F77314.d.70661F740BDBB11AD002FC9110D536F9.e.16")>
<selenium.webdriver.remote.webelement.WebElement (session="dc738d92e277d753d945b6fb1ba3a802", element="f.C8121B418917C39CE0D66451E3F77314.d.70661F740BDBB11AD002FC9110D536F9.e.17")>
<selenium.webdriver.remote.webelement.WebElement (session="dc738d92e277d753d945b6fb1ba3a802", element="f.C8121B418917C39CE0D66451E3F77314.d.70661F740BDBB11AD002FC9110D536F9.e.18")>
<selenium.webdriver.remote.webelement.WebElement (session="dc738d92e277d753d945b6fb1ba3a802", element="f.C8121B418917C39CE0D66451E3F77314.d.70661F740BDBB11AD002FC9110D536F9.e.19")>
<selenium.webdriver.remote.webelement.WebElement (session="dc738d92e277d753d945b6fb1ba3a802", element="f.C8121B418917C39CE0D66451E3F77314.d.70661F740BDBB11AD002FC9110D536F9.e.20")>
<selenium.webdriver.remote.webelement.WebElement (session="dc738d92e277d753d945b6fb1ba3a802", e

## 2